# TX00DQ05-3001 Exercises 4

In [1]:
import numpy as np
import numpy.linalg as LA
import random

## Exercise 1: TD(0) value function estimation

Implement value function estimation for Sutton & Barto example 4.1 with TD(0) algorithm.

In [2]:
# YOUR CODE


## Exercise 2: Implement TD(0) control 

Solve Sutton & Barto example 4.1 with TD(0) control (Sarsa) algorithm. Apply the algorithm to the windy world of Sutton & Barto example 6.5. 

In [3]:
# YOUR CODE


## Exercise 3*: Eligibility trace

Do a random walk in example 4.1 gridworld, and create an eligibility trace from the walk.

*) - not mandatory

In [4]:
# YOUR CODE


## Exercise 4**: TD(lambda)

Implement TD(lambda) algorithm and use it for solving example 6.5. Create a table/plot on the effect of lambda in the performance of the algorithm.

*) - not mandatory

In [5]:
# YOUR CODE
